In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import k3d
import numpy as np
from bmcs_shell.folding.assembly.wb_scanned_cell import WBScannedCell
from bmcs_shell.api import WBTessellation4PEx
from scipy.spatial import cKDTree
import pandas as pd
from openpyxl import load_workbook
import sympy as sp


In [1]:


# Set the reference geometry to be generated from the closed form kinematics
t_num = 30
a_num= 1000/4
b_num= 1615/4
c_num= 645/4
e_num= 286/4
gamma_num= 0.683


wb_shell = WBTessellation4PEx(
                         a=a_num,
                         b = b_num, 
                         c = c_num, 
                         e_x = e_num,
                         gamma=gamma_num, # a value of gamma = 0.75 was estimated from normals, but a CAD comparison showed that 0.75 doesn't lead to closer geometry to the scanned 
                         n_phi_plus=2, # planned 5 
                         n_x_plus=2,  # planned 3
                         wireframe_width=5,
                        ##---- Trimming function works only in WBTessellation4P ----##
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
# wb_shell.interact()
orig_I_Fi = np.copy(wb_shell.I_Fi_trimmed)
orig_X_Ia = np.copy(wb_shell.X_Ia_trimmed)
orig_X_Ib = np.copy(wb_shell.X_Ia_trimmed)


NameError: name 'WBTessellation4PEx' is not defined

# Generate solid geometry 

In [ ]:
def create_solid_connectivity(orig_I_Fi, num_nodes):
    """
    Generates the side faces to connect the original and offset surfaces.
    
    Parameters:
    - orig_I_Fi (numpy.ndarray): (M, K) connectivity matrix of the original surface.
    - num_nodes (int): Number of nodes in the original surface.
    
    Returns:
    - side_I_Fi (numpy.ndarray): Triangulated connectivity matrix for side faces.
    """
    side_faces = []

    for face in orig_I_Fi:
        if len(face) == 3:  # If the original faces are triangles
            a, b, c = face
            a2, b2, c2 = a + num_nodes, b + num_nodes, c + num_nodes

            # Create 3 quads (split into 6 triangles)
            side_faces.append([a, b, b2])
            side_faces.append([a, b2, a2])

            side_faces.append([b, c, c2])
            side_faces.append([b, c2, b2])

            side_faces.append([c, a, a2])
            side_faces.append([c, a2, c2])

    return np.array(side_faces, dtype=np.int32)

def duplicate_and_offset_surface_with_solid(orig_X_Ia, orig_I_Fi, t):
    """
    Duplicates a 3D surface, offsets it in the z-direction by t, and connects both layers to form a solid.
    
    Parameters:
    - orig_X_Ia (numpy.ndarray): (N,3) array of node coordinates.
    - orig_I_Fi (numpy.ndarray): (M,K) connectivity matrix (faces).
    - t (float): Offset distance in the z-direction.
    
    Returns:
    - new_X_Ia (numpy.ndarray): Updated node coordinates (2N,3).
    - solid_I_Fi (numpy.ndarray): Fully closed connectivity matrix.
    """
    num_nodes = orig_X_Ia.shape[0]

    # Duplicate nodes and apply offset in the z-direction
    offset_X_Ia = orig_X_Ia.copy()
    offset_X_Ia[:, 2] += t

    # Combine original and offset nodes
    new_X_Ia = np.vstack((orig_X_Ia, offset_X_Ia))

    # Compute connectivity for the offset surface
    offset_I_Fi = orig_I_Fi + num_nodes  # Shift indices for offset layer

    # Compute side faces to connect both layers
    side_I_Fi = create_solid_connectivity(orig_I_Fi, num_nodes)

    # **Flip the original faces to close the bottom**
    bottom_I_Fi = np.flip(orig_I_Fi, axis=1)  # Flip triangles/quads

    # Combine all faces
    solid_I_Fi = np.vstack((bottom_I_Fi, offset_I_Fi, side_I_Fi))

    return new_X_Ia, solid_I_Fi

def plot_solid_surfaces_in_k3d(geometries, colors=None):
    """
    Plots multiple 3D solid-like structures in k3d with node labels and offset labels to prevent overlap.
    
    Parameters:
    - geometries (list of tuples): Each tuple contains:
        - X_Ia (numpy.ndarray): (N,3) array of node coordinates.
        - I_Fi (numpy.ndarray): (M,K) connectivity matrix (faces).
    - colors (list of int, optional): List of hex colors for each geometry. If None, random colors are used.

    Returns:
    - k3d plot
    """
    plot = k3d.plot()

    num_geometries = len(geometries)
    if colors is None:
        colors = np.random.randint(0, 0xFFFFFF, size=num_geometries).tolist()  # Fixed range

    for idx, (X_Ia, I_Fi) in enumerate(geometries):
        color = colors[idx % len(colors)]  # Assign color

        # Create the solid mesh (filled structure)
        mesh = k3d.mesh(X_Ia.astype(np.float32), I_Fi.astype(np.uint32),
                         color=color, wireframe=False, opacity=1, compression_level=9)
        plot += mesh

        # Add nodes as red points
        points = k3d.points(X_Ia, point_size=20, color=0xff0000)
        plot += points

        # Add node labels with offset to avoid overlap
        for i, position in enumerate(X_Ia):
            offset = np.array([25 * idx, 0, 0])  # Shift each geometry's labels in x-direction
            label_position = position + offset

            label = k3d.text(
                text=str(i),  # Node index as label
                position=label_position.astype(np.float32),
                color=color,  # Black color for contrast
                size=0.7  # Adjust size to prevent clutter
            )
            plot += label

    return plot

# Generate the new geometry with a fully enclosed solid
new_X_Ia, solid_I_Fi = duplicate_and_offset_surface_with_solid(orig_X_Ia, orig_I_Fi, t_num)
new_X_Ia = new_X_Ia - t_num * np.array([0, 0, 1])
new_X_Ib, solid_I_Fi = duplicate_and_offset_surface_with_solid(orig_X_Ib, orig_I_Fi, t_num)

# Example usage
geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
    (new_X_Ib, solid_I_Fi)   # Second geometry
]

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c, 0x3498db])
plot.display()

/Users/gui/Documents/VSCode/bmcs_env/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [ ]:
new_X_Ia[24],new_X_Ia[33]

(array([173.26856995,   0.        , 125.07910156]),
 array([ 4.94768554e+02, -7.62939453e-06,  1.25079102e+02]))

# Understanding the geometry

Given the middle surface between geometry A (red) and geometry B (blue). The geometry A is extruded in $-z$-direction and B in $+z$-direction with thickness $t$.

The following code snippet defines the rotation matrix. Initially it is set to rotate around $x$, but rotation around $y$ and $z$ can be included.


The goal is to calculate the rotation matrix $R(\theta)$, the translation vector $\delta_{\vartheta}$ that brings B to the overlaid state ${\vartheta}$ and the vector $\delta_{\Omega}$ that brings from the overlaid state to the assembled state $\Omega$. Then, this transformation that assembles the waterbomb modules are given by:
$\mathbf{x}_{\Omega} = R(\theta) \cdot \mathbf{x}_{\gamma}+ \delta_{\vartheta} +\delta_{\Omega} $


![Crease convention and nomenclature](analytical-sol-convention.png)

In [ ]:


# Known angles (alpha_x and alpha_z)
theta_x = sp.Symbol('theta_x', real=True)
theta_y = sp.Symbol('theta_y', real=True)  # Unknown
theta_z = sp.Symbol('theta_z', real=True)

# Trig functions
cx = sp.cos(theta_x)
sx = sp.sin(theta_x)
cy = sp.cos(theta_y)
sy = sp.sin(theta_y)
cz = sp.cos(theta_z)
sz = sp.sin(theta_z)

# Rotation matrices
R_x = sp.Matrix([
    [1, 0, 0],
    [0, cx, -sx],
    [0, sx, cx]
])
R_y = sp.Matrix([
    [cy, 0, sy],
    [0, 1, 0],
    [-sy, 0, cy]
])
R_z = sp.Matrix([
    [cz, -sz, 0],
    [sz, cz, 0],
    [0, 0, 1]
])

# Full rotation matrix (Z * Y * X)
#R = R_z @ R_y @ R_x
R = R_x

R

Matrix([
[1,            0,             0],
[0, cos(theta_x), -sin(theta_x)],
[0, sin(theta_x),  cos(theta_x)]])

In [ ]:

# Define symbols
x, y, z, a, b, c, e, gamma, t, delta_sx, delta_sy, delta_sz= sp.symbols(r'x y z a b c e \gamma t \delta_{sx} \delta_{sy} \delta_{sz}', real=True)

X = sp.Matrix([x, y, z])
X

Matrix([
[x],
[y],
[z]])

# Folded coordinates from the kinematics. 

Refer to paper "Description of the origami waterbomb cell kinematics as a basis for the design of thin-walled oricrete shells"

Calculate $x^{\urcorner}$, $y^{\urcorner}$, $z^{\urcorner}$ considering $e$

In [ ]:
# calculate the coordinates for folded state
x_folded = a+e
y_folded = sp.sqrt((b**2*sp.cos(gamma)**2) - a**2 * (1 - sp.sin(gamma))**2) / (sp.cos(gamma))
z_folded = (a * (1 - sp.sin(gamma))) / sp.cos(gamma)

Calculate the angle $\theta$ that governs the tessellation curvature

In [ ]:
# Calculating theta to follow the curvature of the tesselation
theta = ((z_folded - c*sp.cos(gamma))**2- y_folded**2) / ((z_folded - c*sp.cos(gamma))**2+ y_folded**2)
theta = -sp.pi +sp.acos(theta.simplify())
theta 

acos((-a**2*(sin(\gamma) - 1)**2 + b**2*cos(\gamma)**2 - (a*(sin(\gamma) - 1) + c*cos(\gamma)**2)**2)/((-2*a*c*sin(\gamma) + 2*a*c - b**2 - c**2*cos(\gamma)**2)*cos(\gamma)**2)) - pi

In [ ]:
# Substitute parameters
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
}


In [ ]:
theta_num = theta.subs(subs_dict)
theta_num = theta_num.evalf()
theta_np = np.array(sp.lambdify((), theta_num, modules='numpy')()).astype(np.float64).flatten()
theta_np

array([-0.03208749])

In [ ]:
R_num = R.subs(theta_x, theta_num)
R_num = R_num.evalf()
R_np = np.array(sp.lambdify((), R_num, modules='numpy')()).astype(np.float64)
R_np

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.99948524,  0.03208198],
       [ 0.        , -0.03208198,  0.99948524]])

# Enforcing colinearity between the creases of geometry A and B

The global coordinate system is defined between $C_{\gamma \mathrm{A}+}^{\vdash}$ and $C_{\gamma \mathrm{A}+}^{\dashv}$. Basically, the same coordinate already defiend by the WB4CellParamEx().  The symbols $(+)$ and $(-)$ defines if the coordinate belongs to the upper or lower facet of the extruded geometry.

To enforce colinearity between geometry A and B, we fix geometry A and move geometry B. The geometry B is rotated given the tessellation curvature $R(\theta)$ and then translated by the vector $\delta_{\vartheta}$, that overlays geometry B on top of a quarter of geometry A. 

As the rotation is decoupled from the translation, the translation vector $\delta_{\theta}$ is calculated identifying which creases nodes from A have the same position as the crease nodes from B after the overlaying.

When overlaid $\vartheta$, $U_{\vartheta \mathrm{B}-}^{\llcorner} = V_{\mathrm{A}+}^{\dashv} = R(\theta)U_{\vartheta\mathrm{A}+}^{\llcorner} + \delta_{\vartheta}$.

In [ ]:
# Calculate the folded coordinates

Va_gamma_mr_upper = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
Ua_gamma_ll_upper = sp.Matrix([-x_folded, -y_folded, z_folded])

Va_gamma_mr_upper, Ua_gamma_ll_upper

(Matrix([
 [c*sin(\gamma) + e],
 [                0],
 [    c*cos(\gamma)]]),
 Matrix([
 [                                                             -a - e],
 [-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
 [                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [ ]:
# Substitute symbolic expressions with values
Va_gamma_mr_upper_num = Va_gamma_mr_upper.subs(subs_dict)
Ua_gamma_ll_upper_num = Ua_gamma_ll_upper.subs(subs_dict)

# Lambdify (no variables left to evaluate), then evaluate and flatten
Va_gamma_mr_upper_np = np.array(sp.lambdify((), Va_gamma_mr_upper_num, modules='numpy')()).astype(np.float64).flatten()
Ua_gamma_ll_upper_np = np.array(sp.lambdify((), Ua_gamma_ll_upper_num, modules='numpy')()).astype(np.float64).flatten()


In [ ]:
new_X_Ia[23], new_X_Ib[5], Ua_gamma_ll_upper

(array([-321.5       , -385.84936523,  118.88867188]),
 array([-321.5       , -385.84936523,  118.88867188]),
 Matrix([
 [                                                             -a - e],
 [-sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
 [                                    a*(1 - sin(\gamma))/cos(\gamma)]]))

In [ ]:
new_X_Ia[24], new_X_Ib[6], Va_gamma_mr_upper

(array([173.26856995,   0.        , 125.07910156]),
 array([173.26856995,   0.        , 125.07910156]),
 Matrix([
 [c*sin(\gamma) + e],
 [                0],
 [    c*cos(\gamma)]]))

### Calculating $\delta_{\vartheta}$
When overlaid, $\delta_{\vartheta} =  V_{{\gamma} \mathrm{A}+}^{\dashv} - R(\theta)U_{{\gamma} \mathrm{A}+}^{\llcorner}$, given $U_{{\vartheta} \mathrm{B}-}^{\llcorner} = V_{{\gamma} \mathrm{A}+}^{\dashv} $ and  $U_{{\gamma} \mathrm{B}-}^{\llcorner} = U_{{\gamma} \mathrm{A}+}^{\llcorner}$

In [ ]:
delta_theta = Va_gamma_mr_upper - R * Ua_gamma_ll_upper
delta_theta

Matrix([
[                                                                                                                        a + c*sin(\gamma) + 2*e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]])

In [ ]:
# Substitute parameters + theta
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
    theta_x: theta_num
}

In [ ]:
delta_theta_num = delta_theta.subs(subs_dict)
delta_theta_np = np.array(sp.lambdify((), delta_theta_num, modules='numpy')()).astype(np.float64).flatten()
delta_theta_np

array([494.76856909, 381.83659666,  -6.12661375])

In [ ]:
X_theta = R @ X + delta_theta
X_theta

Matrix([
[                                                                                                                                                      a + c*sin(\gamma) + 2*e + x],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + y*cos(theta_x) - z*sin(theta_x) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + y*sin(theta_x) + z*cos(theta_x) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]])

In [ ]:
X_theta = sp.cse(X_theta)
X_theta

([(x0, sin(\gamma)),
  (x1, cos(theta_x)),
  (x2, sin(theta_x)),
  (x3, cos(\gamma)),
  (x4, 1/x3),
  (x5, 1 - x0),
  (x6, a*x4*x5),
  (x7, x4*sqrt(-a**2*x5**2 + b**2*x3**2))],
 [Matrix([
  [                a + c*x0 + 2*e + x],
  [       x1*x7 + x1*y + x2*x6 - x2*z],
  [c*x3 - x1*x6 + x1*z + x2*x7 + x2*y]])])

### Rotating and translating B

Given $R(\theta)$ and $\delta_{\vartheta}$, the geometry B can be overlaid on A

In [ ]:
new_X_Ib_theta = (R_np @ new_X_Ib.T).T + delta_theta_np

In [ ]:
# Example usage
geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
    (new_X_Ib_theta, solid_I_Fi)   # Second geometry
]

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c, 0x3498db])
plot.display()

Output()

In [ ]:
new_X_Ia[18]-new_X_Ib_theta[11], new_X_Ia[24]-new_X_Ib_theta[5]

(array([ 0.00000000e+00, -8.22424380e-06, -6.37251395e-04]),
 array([ 8.52437040e-07, -3.48361436e-05, -5.68853757e-04]))

# Assembling B 


Moving B to assembled state, where the mountain facets are coplanar with the valley facets and there is no gap between them.

In assembled state $\Omega$:

- 1. condition: $M_{\gamma \mathrm{A}+}^{\urcorner}$ and $V_{\gamma\mathrm{A}+}^{\dashv}$ are colinear
- 2. condition: $C_{\gamma \mathrm{A}-}^{\dashv}$ and $U_{\gamma \mathrm{A}-}^{\urcorner}$ are colinear
- 3. condition: $C_{\gamma \mathrm{A}-}^{\dashv}$ are $V_{\vartheta \mathrm{B}+}^{\vdash}  $ are colinear

Given the rotation $R(\theta)$ and $\delta_{\vartheta}$ applied to geometry B, we need to compute a translation vector $\delta_{\Omega}$ that will shift the geometry to the assembled position


![Assembley analysis](assembly-analysis.png)

### Validating the computed crease nodes

Calculate: $M_{\gamma\mathrm{A}+}^{\urcorner}$   

In [ ]:
Ma_gamma_ur_upper = R @ sp.Matrix([-e, 0, 0]) + delta_theta
Ma_gamma_ur_upper

Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]])

In [ ]:
# thickness vector
t_v = sp.Matrix([0, 0, t])
t_v

Matrix([
[0],
[0],
[t]])

In [ ]:
# Substitute parameters + theta
Ma_gamma_ur_upper_num = Ma_gamma_ur_upper.subs(subs_dict)
Ma_gamma_ur_upper_np = np.array(sp.lambdify((), Ma_gamma_ur_upper_num, modules='numpy')()).astype(np.float64).flatten()
new_X_Ia[32],  Ma_gamma_ur_upper_np

(array([423.26856909, 381.83660889,  -6.12597656]),
 array([423.26856909, 381.83659666,  -6.12661375]))

Calculate: $C_{\gamma\mathrm{A}-}^{\dashv}$

In [ ]:
Ca_gamma_mr_lower = sp.Matrix([e, 0, 0]) - t_v
Ca_gamma_mr_lower

Matrix([
[ e],
[ 0],
[-t]])

In [ ]:
Ca_gamma_mr_lower_num = Ca_gamma_mr_lower.subs(subs_dict)
Ca_gamma_rl_lower_np = np.array(sp.lambdify((), Ca_gamma_mr_lower_num, modules='numpy')()).astype(np.float64).flatten()
new_X_Ia[0],  Ca_gamma_rl_lower_np

(array([ 71.5,   0. , -30. ]), array([ 71.5,   0. , -30. ]))

Calculate: $U_{\gamma\mathrm{A}-}^{\urcorner}$

In [ ]:
Ua_gamma_ur_lower = sp.Matrix([x_folded, y_folded, z_folded]) - t_v
Ua_gamma_ur_lower

Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                               a*(1 - sin(\gamma))/cos(\gamma) - t]])

In [ ]:
Ua_gamma_ur_lower_num = Ua_gamma_ur_lower.subs(subs_dict)
Ua_gamma_ur_lower_np = np.array(sp.lambdify((), Ua_gamma_ur_lower_num, modules='numpy')()).astype(np.float64).flatten()
new_X_Ia[2],  Ua_gamma_ur_lower_np

(array([321.5       , 385.84936523,  88.88867188]),
 array([321.5       , 385.849374  ,  88.88785926]))

Calculate: $V_{\gamma\mathrm{A}+}^{\dashv}$

In [ ]:
Va_gamma_mr_upper = sp.Matrix([e+c*sp.sin(gamma), 0, c*sp.cos(gamma)])
Va_gamma_mr_upper

Matrix([
[c*sin(\gamma) + e],
[                0],
[    c*cos(\gamma)]])

In [ ]:
Va_gamma_mr_upper_num = Va_gamma_mr_upper.subs(subs_dict)
Va_gamma_mr_upper_np = np.array(sp.lambdify((), Va_gamma_mr_upper_num, modules='numpy')()).astype(np.float64).flatten()
new_X_Ia[24],  Va_gamma_mr_upper_np

(array([173.26856995,   0.        , 125.07910156]),
 array([173.26856909,   0.        , 125.0788585 ]))

Calculate: $V_{\vartheta\mathrm{B}+}^{\vdash}$

In [ ]:
Vb_theta_ml_upper = R @ (sp.Matrix([-e-c*sp.sin(gamma), 0, c*sp.cos(gamma)]) + t_v)+ delta_theta
Vb_theta_ml_upper

Matrix([
[                                                                                                                                                                             a + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma) - (c*cos(\gamma) + t)*sin(theta_x)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma) + (c*cos(\gamma) + t)*cos(theta_x)]])

In [ ]:
Vb_theta_ml_upper_num = Vb_theta_ml_upper.subs(subs_dict)
Vb_theta_ml_upper_np = np.array(sp.lambdify((), Vb_theta_ml_upper_num, modules='numpy')()).astype(np.float64).flatten()
new_X_Ib_theta[25],  Vb_theta_ml_upper_np

(array([321.49999915, 386.81184118, 148.87265942]),
 array([321.5       , 386.81183338, 148.87241649]))

# Defining crease vectors

In assembled state $\Omega$: 

- The geometry B is shifted from the overlaid state to the assembled state by a scalar $s$ in the direction of $\mathbf{e}_{\mathrm{A+}}$. Thus, the translation vector $\delta_{\Omega} = s \cdot \mathbf{e}_{\mathrm{a+}}$.
- $s$ is obtained under the condition that $\mathbf{e}_{\mathrm{A-}}$ and $\mathbf{u}_{\mathrm{AB}}$ became colinear after the shift $\delta_{\Omega}$ in the assembled state $\Omega$.

$\mathbf{e}_{\mathrm{A+}} = \frac{M_{\gamma \mathrm{A}+}^{\urcorner} - V_{\gamma \mathrm{A}+}^{\dashv}}{\lVert M_{\gamma \mathrm{A}+}^{\urcorner} - V_{\gamma \mathrm{A}+}^{\dashv} \rVert}$



In [ ]:
n_upper = (Ma_gamma_ur_upper - Va_gamma_mr_upper) / (Ma_gamma_ur_upper - Va_gamma_mr_upper).norm()
n_upper

Matrix([
[                                                                                                                                a/sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2)],
[ (a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))/sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2)],
[(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))/sqrt(a**2 + Abs(-a*(1 

$\mathbf{e}_{\mathrm{A-}}= \frac{U_{\gamma \mathrm{A}-}^{\urcorner} - C_{\gamma \mathrm{A}-}^{\dashv}}{\lVert U_{\gamma \mathrm{A}-}^{\urcorner} - C_{\gamma \mathrm{A}-}^{\dashv} \rVert}$

In [ ]:
n_lower = (Ua_gamma_ur_lower - Ca_gamma_mr_lower) / (Ua_gamma_ur_lower - Ca_gamma_mr_lower).norm()
n_lower

Matrix([
[                                                                   a/sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma))],
[                                   a*(1 - sin(\gamma))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma))]])

$\mathbf{u}_{\mathrm{AB}} = V_{\vartheta \mathrm{B}+}^{\vdash} - C_{\gamma \mathrm{A}-}^{\dashv}$

In [ ]:
n_ab = (Vb_theta_ml_upper - Ca_gamma_mr_lower)
n_ab

Matrix([
[                                                                                                                                                                                     a],
[                     a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma) - (c*cos(\gamma) + t)*sin(theta_x)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + t + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma) + (c*cos(\gamma) + t)*cos(theta_x)]])

In [ ]:
cross_nu_nl = n_upper.cross(n_lower)
cross_nu_nl

Matrix([
[a*(1 - sin(\gamma))*(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2)*cos(\gamma)) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))*

In [ ]:
cross_ab_nl = n_ab.cross(n_lower)
cross_ab_nl

Matrix([
[a*(1 - sin(\gamma))*(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma) - (c*cos(\gamma) + t)*sin(theta_x))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma)) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + t + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma) + (c*cos(\gamma) + t)*cos(theta_x))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma))],
[                                                                                                                                                                                                                                   -a**2*(1 - sin(\gamma))/(sqrt(a**2 + Abs(sqrt(

In [ ]:
norm_cross_sq = cross_nu_nl.dot(cross_nu_nl)
norm_cross_sq

(a*sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2)*cos(\gamma)) - a*(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gam

In [ ]:
s_scalar = - cross_ab_nl.dot(cross_nu_nl) / norm_cross_sq
s_scalar

(-(a*sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma)) - a*(a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma) - (c*cos(\gamma) + t)*sin(theta_x))/sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2))*(a*sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma))**2)*cos(\gamma)) - a*(a*(1

In [ ]:
delta_omega = s_scalar * n_upper
delta_omega_cse = sp.cse(delta_omega)
delta_omega_cse

([(x0, a**2),
  (x1, sin(theta_x)),
  (x2, sin(\gamma) - 1),
  (x3, cos(\gamma)),
  (x4, 1/x3),
  (x5, a*x4),
  (x6, x2*x5),
  (x7, cos(theta_x)),
  (x8, -x2),
  (x9, sqrt(b**2*x3**2 - x0*x8**2)),
  (x10, x4*x9),
  (x11, x10*x7),
  (x12, x5*x8),
  (x13, x12*x7),
  (x14, x1*x4*x9 - x13),
  (x15, 1/sqrt(x0 + Abs(x14)**2 + Abs(x1*x6 - x11)**2)),
  (x16, 1/sqrt(x0 + Abs(x10)**2 + Abs(x6)**2)),
  (x17, x0*x16*x4*x8),
  (x18, a*x14*x15*x16 - x15*x17),
  (x19, a*x16),
  (x20, x15*x19),
  (x21, x1*x12 + x11),
  (x22, x10*x20 - x20*x21),
  (x23, x12*x16),
  (x24, x10*x16),
  (x25, -x14*x15*x24 + x15*x21*x23),
  (x26, c*x3 + t),
  (x27, x1*x10 - x13 + x26*x7 + x26),
  (x28, -x1*x26 + x21),
  (x29,
   x15*(-x18*(a*x16*x27 - x17) - x22*(x10*x19 - x19*x28) - x25*(x23*x28 - x24*x27))/(x18**2 + x22**2 + x25**2))],
 [Matrix([
  [  a*x29],
  [x21*x29],
  [x14*x29]])])

In [ ]:
# list variables of delta_omegaega
delta_omega_vars = delta_omega.free_symbols
delta_omega_vars

{\gamma, a, b, c, t, theta_x}

In [ ]:
# Substitute parameters + theta
subs_dict = {
    a: a_num,
    b: b_num,
    c: c_num,
    e: e_num,
    gamma: gamma_num,
    t: t_num,
    theta_x: theta_num,
}
delta_omega_num = delta_omega.subs(subs_dict)
delta_omega_num = delta_omega_num.evalf()
delta_omega_np = np.array(sp.lambdify((), delta_omega_num, modules='numpy')()).astype(np.float64).flatten()
delta_omega_np

array([ 59.96217178,  91.58300641, -31.46946026])

In [ ]:
s_scalar_num = s_scalar.subs(subs_dict)
s_scalar_num = s_scalar_num.evalf()
s_scalar_np = np.array(sp.lambdify((), s_scalar_num, modules='numpy')()).astype(np.float64).flatten()
s_scalar_np

array([113.9001143])

In [ ]:
new_X_Ib_shift = new_X_Ib_theta + delta_omega_np


In [ ]:


# Example usage
geometries = [
    (new_X_Ia, solid_I_Fi),  # First geometry
    (new_X_Ib_shift, solid_I_Fi)   # Second geometry
]

plot = plot_solid_surfaces_in_k3d(geometries, colors=[0xe74c3c, 0x3498db])
plot.display()

Output()

# Slit depth

The slit depth for a compatible assembly is calculated based on similar triangles:

so the slit depth $L_s = \frac{\left\| V_{\gamma A+}^{\dashv}-U_{\gamma A+}^{\urcorner}\right\|\cdot \left\| U_{\Omega B-}^{\llcorner}-M_{\gamma A+}^{\urcorner}\right\|}{2\left\|V_{\gamma A+}^{\dashv}-M_{\gamma A+}^{\urcorner} \right\|}$

In [ ]:
Va_gamma_mr_upper 

Matrix([
[c*sin(\gamma) + e],
[                0],
[    c*cos(\gamma)]])

In [ ]:
Ua_gamma_ur_upper =  sp.Matrix([x_folded, y_folded, z_folded])
Ua_gamma_ur_upper

Matrix([
[                                                             a + e],
[sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma)],
[                                   a*(1 - sin(\gamma))/cos(\gamma)]])

In [ ]:
l2 = Va_gamma_mr_upper - Ua_gamma_ur_upper
l2 = l2.norm()
l2

sqrt((a - c*sin(\gamma))**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(-a*(1 - sin(\gamma))/cos(\gamma) + c*cos(\gamma))**2)

In [ ]:
l2_num = l2.subs(subs_dict)
l2_num = l2_num.evalf()
l2_num = np.array(sp.lambdify((), l2_num, modules='numpy')()).astype(np.float64).flatten()
l2_num

array([413.38919313])

In [ ]:
Ub_omega_ll_lower = R @( Ua_gamma_ll_upper) +  delta_theta + delta_omega


In [ ]:
Ma_gamma_ur_upper

Matrix([
[                                                                                                                          a + c*sin(\gamma) + e],
[                 a*(1 - sin(\gamma))*sin(theta_x)/cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta_x)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta_x)/cos(\gamma)]])

In [ ]:
a1 = Ub_omega_ll_lower - Ma_gamma_ur_upper
a1 = a1.norm()
a1_num = a1.subs(subs_dict)
a1_num = a1_num.evalf()
a1_num = np.array(sp.lambdify((), a1_num, modules='numpy')()).astype(np.float64).flatten()
a1_num

array([360.98309503])

In [ ]:
a2 = Va_gamma_mr_upper - Ma_gamma_ur_upper
a2 = a2.norm()
a2_num = a2.subs(subs_dict)
a2_num = a2_num.evalf()
a2_num = np.array(sp.lambdify((), a2_num, modules='numpy')()).astype(np.float64).flatten()
a2_num

array([474.88320933])

In [ ]:
l1_num = l2_num*a1_num/a2_num
l1_num


array([314.23833789])

In [ ]:
Ls=l1_num/2
Ls

array([157.11916894])

In [ ]:
Ls = (l2*a1)/(2*a2)
Ls

sqrt((a - c*sin(\gamma))**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(-a*(1 - sin(\gamma))/cos(\gamma) + c*cos(\gamma))**2)*sqrt(Abs(a + a*((a*sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*cos(\gamma)) + a*(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x)/cos(\gamma) + (c*cos(\gamma) + t)*sin(theta_x))/sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2))*(a*sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/(sqrt(a**2 + Abs(sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)/cos(\gamma))**2 + Abs(a*(sin(\gamma) - 1)/cos(\gamma))**2)*sqrt(a**2 + Abs(a*(sin(\gamma) - 1)*sin(theta_x)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta_x

In [ ]:
Ls.free_symbols

{\gamma, a, b, c, t, theta_x}

In [ ]:
Ls_num = Ls.subs(subs_dict)
Ls_num = Ls_num.evalf()
Ls_num = np.array(sp.lambdify((), Ls_num, modules='numpy')()).astype(np.float64).flatten()
Ls_num



array([157.11916894])